In this notebook, you will learn to add AI options to a basic Teams-ai bot.

A Teams-ai application object can have an `ApplicationOptions.ai` property with `AIOption` type. A bot application with `ApplicationOptions.ai` property can work as a bot with AI abilities. For example, it can give a artificial response to your input according to prompts.

`AIOptions` consists of `planner`, `prompt` and `history`. The `prompt` determines what prompts to use. The `planner` will generate plan according to given prompts and chat history. We will show you how `prompt` and  `planner` work.

## Prompt
In most cases, you'll want to create your prompts in a separate file so you can easily import them into Teams-ai across multiple projects and share them with others. 

In this section, we'll demonstrate how to create the files necessary for a prompt so you can easily use them in `AIOption`.

Here is a sample code snippet about how to generate a prompts home folder `src/prompts`, and a named prompt folder `src/prompts/chat`. In this example, the prompt will be called `'chat'`. Once inside of a prompts folder, you'll need to create two new files `config.json` and `skprompt.txt`. The `skprompt.txt` file contains the prompt that will be sent to the AI service and the `config.json` file contains the configuration along with semantic descriptions that can be used by planners.
```
Prompts
│
└─── chat
     |
     └─── config.json
     └─── skprompt.txt
```

> We have already prepare the files in [src/prompts/chat](./src/prompts/chat/). In a real scenario, you would need to manually manage the prompts folder following the file naming conventions. Please refer [semantic kernel doc](https://learn.microsoft.com/en-us/semantic-kernel/prompts/saving-prompts-as-files?tabs=python) for more information.

### Prompt actions

Plans let the model perform actions or respond to the user. You can create a schema of the plan and add a list of actions that you support to perform an action and pass arguments. The OpenAI endpoint figures out the actions required to be used, extracts all the entities, and passes those as arguments to the action call. A Teams-ai application with `AIOptions` will always generate a plan with either `"SAY"` type commands or `"DO"` type commands. 
* `SAY`: `SAY` commands will give a response text by calling OpenAI according to user's input.
* `DO`: `DO` commands will map action names with entities and call action methods. We use annotation like `@app.ai.action("<action_name>")` to hook up methods with action names.

The language supports features that allow you to include variables, call external functions, and pass parameters to functions. You don't need to write any code or import any external libraries, just use the curly braces {{...}} to embed expressions in your prompts.
* `{{function}}`: Calls a registered function and inserts its return value string. We use annotation like `@app.ai.function("<function_name>")` to hook up methods with function names.
* `{{$input}}`: Inserts the message text. It gets its value from state.temp.input.
* `{{$history}}`: Inserts the history. It gets its value from state.temp.history.

> For more information of prompt syntax in Teams-ai, please refer to [this doc](https://learn.microsoft.com/en-us/microsoftteams/platform/bots/how-to/teams%20conversational%20ai/how-conversation-ai-get-started?tabs=javascript4%2Cjavascript1%2Cjavascript3%2Cjavascript2#prompt).

> The following text content shows a prompt template that defines some lights operating actions. We will introduce how the prompt with different `DO` and `SAY` commands work in the "Add AIOptions to a bot" [section](#add-aioptions-to-a-bot).

In [ ]:
skprompt_txt="""The following is a conversation with an AI assistant. 
The assistant can turn a light on or off.
The assistant must return the following JSON structure:

{"type":"plan","commands":[{"type":"DO","action":"<name>","entities":{"<name>":<value>}},{"type":"SAY","response":"<response>"}]}

The following actions are supported:

- LightsOn
- LightsOff

Always respond in the form of a JSON based plan. Stick with DO/SAY."""

## Planner
In this section, we will introduce how does a `planner` generates a plan.

1. First, we import needed packages

In [ ]:
# import modules from src
import src.config as config
from src.bot import *
from teams import AIHistoryOptions, AzureOpenAIPlanner, AzureOpenAIPlannerOptions, ConversationHistory, ConversationState, TempState, UserState
from botbuilder.schema import ChannelAccount, ConversationAccount
from unittest.mock import MagicMock 

2. Second, we initiate a `planner` object with Azure OpenAI keys and values, determining prompt home folder.

    > Please config your **AZURE_OPENAI_KEY**, **AZURE_OPENAI_MODEL_DEPLOYMENT_NAME** and **AZURE_OPENAI_ENDPOINT** in `.env` file.

In [ ]:
# register a planner
planner = AzureOpenAIPlanner(
    AzureOpenAIPlannerOptions(
        config.AZURE_OPENAI_KEY,
        config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME,
        config.AZURE_OPENAI_ENDPOINT,
        prompt_folder="src/prompts",
    )
)
print(f"AZURE_OPENAI_KEY={config.AZURE_OPENAI_KEY}")
print(f"AZURE_OPENAI_MODEL_DEPLOYMENT_NAME={config.AZURE_OPENAI_MODEL_DEPLOYMENT_NAME}")
print(f"AZURE_OPENAI_ENDPOINT={config.AZURE_OPENAI_ENDPOINT}")

3. Then, we mock a `TurnContext` object and a `TurnState` object. They will be used when generating a plan. We already defined them in [src/utils/mockConstants.py](src/utils/mockConstants.py).

    > In this example, we set `TempState.input='hi'` to emulate that user inputs 'hi' as the input text.

In [ ]:
from src.utils.mockConstants import *

print(f"context={context}")
print(f"state.temp={state.temp}")

4. Last, we call `planner._prompt_manager.render_prompt()` and `planner.generate_plan()` to see what prompt content passed to planner and what plan is generated by planner. 

    > In this example, we choose `"chat"` as the prompt template name to generate a prompt.

In [ ]:

# render prompt
prompt_name="chat"
prompt = await planner._prompt_manager.render_prompt(context, state, prompt_name)
print(f"prompt:\n{prompt.text}\n")

# generate plan
plan = await planner.generate_plan(
    context, state, prompt_name, history_options=AIHistoryOptions(assistant_history_type="text")
)
print(f"plan:\n{plan}")

## Add AIOptions to a bot
Now we know how to manage the prompts folder and how to set up a planner. We can initiate a Teams-ai bot application with `AIOptions` with `prompt` and `planner` we need. We will show you how does an application with "chatLight" as prompt in AIOptions work.

First, we generate a prompt template named "chatLight" with `DO` and `SAY` commands.

> We have already prepare the files in [src/prompts/chatLight](./src/prompts/chatLight/), and the following code will copy them into `src/prompts/`. In a real scenario, you would need to manually manage the prompts folder following the file naming conventions.

Then we create an Application with `prompt="chatLight"` in `AIOptions`.

In [ ]:
from botbuilder.core import BotFrameworkAdapterSettings, MemoryStorage
from teams import AIOptions, Application, ApplicationOptions, TurnState
storage = MemoryStorage()
app = Application[TurnState](
    ApplicationOptions(
        auth=BotFrameworkAdapterSettings(
            app_id=config.app_id,
            app_password=config.app_password,
        ),
        ai=AIOptions(
            planner=planner,
            prompt="chatLight",
            history=AIHistoryOptions(assistant_history_type="text"),
        ),
        storage=storage,
    )
)

Then we need to define actions.

In [ ]:
# MyActionTurnContext = ActionTurnContext[Dict[str, Any]]

@app.ai.action("LightsOn")
async def on_lights_on(context, state):
    print("[lights on]")
    return True


@app.ai.action("LightsOff")
async def on_lights_off(context, state):
    print("[lights off]")
    return True

Start the bot using `Flask` server. After starting the server, you can try it with [Teams App Test Tool](https://learn.microsoft.com/en-us/microsoftteams/platform/toolkit/debug-your-teams-app-test-tool). You only need to run 2 commands if you have installed `Node` in your system.
```bash
npm install -g @microsoft/teams-app-test-tool
teamsapptester start
```

In [ ]:
from flask import Flask, request, jsonify
from botbuilder.schema import Activity

# Define the Flask API
api = Flask(__name__)
# Define the route for receiving messages
@api.route('/api/messages', methods=['POST'])
async def on_messages():
    activity = Activity().deserialize(request.json)

    auth_header = request.headers['Authorization'] if 'Authorization' in request.headers else ''
    response = await app.process_activity(activity, auth_header)
    
    if response:
        return jsonify(response.body), response.status
    return '', 200

# Run the Flask API
if __name__ == "__main__":
    try:
        api.run(host='localhost', port=config.port)
    except Exception as error:
        raise error

When you successfully install Teams App Test Tool and start it in a browser, you will see the test window. Now you can test your echo bot!
![Alt text](src/utils/image.png)